In [84]:
import os
from dotenv import load_dotenv

# importing the libarires for loading the data and creating the document nodes
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader, SummaryIndex, VectorStoreIndex

# For defining the llm model and embeddings to use
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# For creating the query engine tools
from llama_index.core.tools import QueryEngineTool

# For creating the query engine routers
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector

In [63]:
openai_api_key = os.getenv("OPENAI_API_KEY")
load_dotenv()

if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("OpenAI API key has been set.")
else:
    print("Error: OpenAI API key not found in .env file.")

OpenAI API key has been set.


# Loading the PDF document

In [64]:
document_path = '/Users/ishadave/Documents/Priyesh/Repositories/Generative AI/langchain-projects/Advance RAG/Datasets/'
directory_reader_obj = SimpleDirectoryReader(document_path)
loaded_document = directory_reader_obj.load_data()
loaded_document

[Document(id_='efceee4f-d0a1-471e-9e50-a6054165c48d', embedding=None, metadata={'page_label': '1', 'file_name': 'attention.pdf', 'file_path': '/Users/ishadave/Documents/Priyesh/Repositories/Generative AI/langchain-projects/Advance RAG/Datasets/attention.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2024-07-05', 'last_modified_date': '2024-07-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip

In [65]:
len(loaded_document)

15

In [66]:
loaded_document[0].metadata

{'page_label': '1',
 'file_name': 'attention.pdf',
 'file_path': '/Users/ishadave/Documents/Priyesh/Repositories/Generative AI/langchain-projects/Advance RAG/Datasets/attention.pdf',
 'file_type': 'application/pdf',
 'file_size': 2215244,
 'creation_date': '2024-07-05',
 'last_modified_date': '2024-07-05'}

# Creating the nodes from the loaded document

In [67]:
sentence_splitter = SentenceSplitter(chunk_size=1000,
                                     chunk_overlap=200)

In [68]:
document_nodes = sentence_splitter.get_nodes_from_documents(loaded_document)
print(f'Splitted the documents in {len(document_nodes)} nodes')

Splitted the documents in 17 nodes


In [69]:
document_nodes

[TextNode(id_='31cce7f8-062e-41e4-88f8-613c37f66cf8', embedding=None, metadata={'page_label': '1', 'file_name': 'attention.pdf', 'file_path': '/Users/ishadave/Documents/Priyesh/Repositories/Generative AI/langchain-projects/Advance RAG/Datasets/attention.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2024-07-05', 'last_modified_date': '2024-07-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='efceee4f-d0a1-471e-9e50-a6054165c48d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'attention.pdf', 'file_path': '/Users/ishadave/Documents/Priyesh/Repositories/Generative AI/langchain-projects/Advance RAG/Datasets/attention.pdf', 'file_type': 'appli

# Defining the OpenAI model and embeddings

In [100]:
llm_model = OpenAI('gpt-3.5-turbo')

In [99]:
llm_embeddings = OpenAIEmbedding(model='text-embedding-ada-002')

# Creating summary and vector indexes

In [72]:
summary_index = SummaryIndex(document_nodes)
summary_index

In [73]:
vector_index =  VectorStoreIndex(document_nodes)
vector_index

# Defining the query engines

In [74]:
summary_query_engine = summary_index.as_query_engine(use_async=True)
summary_query_engine

In [75]:
vector_query_engine = vector_index.as_query_engine()
vector_query_engine

# Defining the tools

In [76]:
summary_tool = QueryEngineTool.from_defaults(query_engine = summary_query_engine,
                               description = 'This tool if for summarization task')
summary_tool

In [77]:
vector_tool = QueryEngineTool.from_defaults(query_engine = vector_query_engine,
                               description = 'This tool if for vector search task form the indexes')
vector_tool

# Defining the routers

In [96]:
router = RouterQueryEngine(selector = LLMSingleSelector.from_defaults(),
                           query_engine_tools = [summary_tool, vector_tool],
                           verbose = True)
router

In [91]:
response = router.query("what is the summary of this paper?")
print(response)

Selecting query engine 0: This tool is specifically designed for summarization tasks, making it the most relevant choice for summarizing a paper..
The paper introduces the Transformer model, a network architecture based solely on attention mechanisms, eliminating the need for recurrent or convolutional layers. The Transformer model excels in machine translation tasks, achieving state-of-the-art results on the WMT 2014 English-to-German and English-to-French translation tasks. It is highly parallelizable, faster to train, and outperforms previous models while requiring less training time. The paper also discusses the model's components like multi-head attention, positional encodings, and self-attention. Additionally, the Transformer model is shown to generalize well to tasks like English constituency parsing.


In [90]:
response = router.query("what is the attention is all you need?")
print(response)

Selecting query engine 0: This tool is for summarization task, which involves paying attention to important information and condensing it into a shorter form..
The attention in the context provided refers to the attention mechanism used in the Transformer model, which is a network architecture based solely on attention mechanisms. This attention mechanism allows the model to draw global dependencies between input and output sequences without the need for recurrent or convolutional layers. It enables the model to focus on different parts of the input sequence when generating the output, improving the quality of the translation tasks it performs.


In [89]:
response = router.query("Who wrote this paper?")
print(response)

Selecting query engine 1: The tool mentioned is related to vector search task, which could potentially be used to search for the author of a paper based on indexes..
Vinyals & Kaiser, Koo, Petrov, Sutskever, and Hinton.


In [87]:
response = router.query("Who wrote this paper? use the query tool not summarization")
print(response)

Selecting query engine 1: The tool mentioned is specifically for vector search task from the indexes, which is more relevant to the question of finding the author of a paper..
Vinyals & Kaiser, Koo, Petrov, Sutskever, and Hinton.


In [97]:
response = router.query("Give the names of the writers of this paper")
print(response)

Selecting query engine 0: The tool is designed for summarization tasks, which would be relevant for extracting the names of the writers from the paper..
Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin


In [98]:
response.metadata

{'31cce7f8-062e-41e4-88f8-613c37f66cf8': {'page_label': '1',
  'file_name': 'attention.pdf',
  'file_path': '/Users/ishadave/Documents/Priyesh/Repositories/Generative AI/langchain-projects/Advance RAG/Datasets/attention.pdf',
  'file_type': 'application/pdf',
  'file_size': 2215244,
  'creation_date': '2024-07-05',
  'last_modified_date': '2024-07-05'},
 '9f93b2b2-cadf-48a8-a91a-670c2f72f0dd': {'page_label': '2',
  'file_name': 'attention.pdf',
  'file_path': '/Users/ishadave/Documents/Priyesh/Repositories/Generative AI/langchain-projects/Advance RAG/Datasets/attention.pdf',
  'file_type': 'application/pdf',
  'file_size': 2215244,
  'creation_date': '2024-07-05',
  'last_modified_date': '2024-07-05'},
 '82a216e6-4a17-4cd8-ab00-370dc215d3d2': {'page_label': '3',
  'file_name': 'attention.pdf',
  'file_path': '/Users/ishadave/Documents/Priyesh/Repositories/Generative AI/langchain-projects/Advance RAG/Datasets/attention.pdf',
  'file_type': 'application/pdf',
  'file_size': 2215244,
  'c